In [6]:
import httpx
from rich import print

OAI = "https://www.radar-service.eu/oai/OAIHandler?verb=ListRecords&metadataPrefix=oai_dc&set=radar4culture"
r = httpx.get(OAI)
if r.status_code == 200:
    data = r.text
    print(len(data))
print(data[:2000])

41034

<?xml version="1.0" encoding="UTF-8" ?><?xml-stylesheet type="text/xsl" href="/oai/stylesheet"?><OAI-PMH 
xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ 
http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd"><responseDate>2023-04-21T14:40:12Z</responseDate><request 
verb="ListRecords" metadataPrefix="oai_dc" 
set="radar4culture">https://www.radar-service.eu/oai/OAIHandler</request><ListRecords>
<record><header><identifier>10.22000/708</identifier><datestamp>2022-08-25T10:46:33Z</datestamp><setSpec>radar4cult
ure</setSpec></header><metadata><oai_dc:dc xmlns:dc="http://purl.org/dc/elements/1.1/"
           xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/"
           xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
           xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/oai_dc/ 
https://www.openarchives.org/OAI/2.0/oai_dc.xsd http://purl.org/dc/elements/1.1/ 
https://www.radar-service.eu/schemas/dc.xsd">
   <dc:identifier>10.22000/708</dc:identifier>
   <dc:identifier>DOI</dc:identifier>
   <dc:creator>Zentrum für Geschichte der Psychologie der Universität Würzburg</dc:creator>
   <dc:title>Stereoskop hemisphärisch</dc:title>
   <dc:publisher>Zentrum für Geschichte der Psychologie</dc:publisher>
   <dc:date>2022</dc:date>
   <dc:subject>Psychology</dc:subject>
   <dc:type/>
   <dc:type>Image</dc:type>
   <dc:rights>CC BY-SA 4.0 Attribution-ShareAlike</dc:rights>
   <dc:rights>Zentrum für Geschichte der Psychologie der Universität Würzburg</dc:rights>
   <dc:description>Hersteller Underwood &amp; Underwood, USA</dc:description>
   <dc:language/>
   <dc:format>application/zip</dc:format>
</oai_dc:dc></metadata></record>
<record><header><identifier>10.22000/714</identifier><datestamp>2022-08-25T10:46:38Z</datestamp><setSpec>radar4cult
ure</setSpec></header><metadata><oai_dc:dc xmlns:dc="http://purl.org/dc/elements/1.1/"
           xmlns:oai_dc="http://w

In [7]:
import xml.etree.ElementTree as ET

# Parse the XML string into an ElementTree object
root = ET.fromstring(data)

# Extract all record elements
records = root.findall('.//{http://www.openarchives.org/OAI/2.0/}record')

# Loop through the records and extract the metadata values
buf = []
for record in records:
    metadata = record.find('.//{http://www.openarchives.org/OAI/2.0/}metadata/{http://www.openarchives.org/OAI/2.0/oai_dc/}dc')
    record_dict = {}
    for element in metadata:
        key = element.tag.replace('{http://purl.org/dc/elements/1.1/}', '') # remove namespace from tag name
        value = element.text
        if value is not None:
            record_dict.setdefault(key, []).append(value)
    buf.append(record_dict)


In [8]:
print(buf[0])

{
    'identifier': ['10.22000/708', 'DOI'],
    'creator': ['Zentrum für Geschichte der Psychologie der Universität Würzburg'],
    'title': ['Stereoskop hemisphärisch'],
    'publisher': ['Zentrum für Geschichte der Psychologie'],
    'date': ['2022'],
    'subject': ['Psychology'],
    'type': ['Image'],
    'rights': [
        'CC BY-SA 4.0 Attribution-ShareAlike',
        'Zentrum für Geschichte der Psychologie der Universität Würzburg'
    ],
    'description': ['Hersteller Underwood & Underwood, USA'],
    'format': ['application/zip']
}

We can see that this XML fragment has very sparse metadata, and there are no IRIs used to map the values to controlled vocabularies.

It is non-trivial to use the [RADAR API](https://radar.products.fiz-karlsruhe.de/de/radarfeatures/radar-api) - there it is necssary to register a user and supply tokens to make the calls.

Does it contain more/better mappings, or should we somehow have an _external_ reconciliation of the strings to IRIs form the OAI-PMH data?